In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
sns.set(color_codes=True)
%matplotlib inline
df_all = pd.read_csv('C:\\Users\\Brend\\Downloads\\correlation_top42_v2.csv', index_col = 'name', encoding = "UTF-8")
print("Total: %d authors" % df_all.shape[0])

Total: 1965872 authors


In [3]:
df_trimmed = df_all[df_all['num_pubs'] >= 50]
df_trimmed = df_trimmed[df_trimmed['TotalConns'] >= 25]
print("Remaining after trimming: %d authors" % df_trimmed.shape[0])

Remaining after trimming: 368618 authors


In [4]:
linear_metrics = ['pct_collab_linauth', 'pct_collab_linprof', 
                  'pct_collab_sqrtauth', 'pct_collab_sqrtprof', 
                  'pct_collab_unweighted']
log_metrics = []
all_metrics = linear_metrics + log_metrics
impact = ['avg_PR', 'max_PR', 'avg_AR', 'max_AR']


In [5]:
df_log = df_trimmed.copy()
for col in df_trimmed.columns:
    log_col = df_log[[col]]
    df_log[col + '_log'] = log_col.apply(np.log, axis=0)
df_log = df_log[[col for col in df_log.columns if 'log' in col]]

In [6]:
from sklearn.preprocessing import StandardScaler
df_scaled = df_trimmed.copy()
for col in df_trimmed.columns:
    ss = StandardScaler()
    scaled_col = df_scaled[[col]]
    df_scaled[col + '_zscore'] = ss.fit_transform(scaled_col)
df_zscores = df_scaled[[col for col in df_scaled.columns if 'zscore' in col]]

In [7]:
import numpy as np
from sklearn.linear_model import LinearRegression
print('r2 scores:')
d = dict()
for x_col in linear_metrics:
    score_list = []
    index_list = []
    for y_col in impact:
        X = df_zscores[[x_col + '_zscore']]
        y = df_log[[y_col + '_log']]
        reg = LinearRegression().fit(X, y)
        score = round(reg.score(X, y), 4)
        score_list.append(score)
        index_list.append(y_col)
    d[x_col + '_zscore'] = pd.Series(score_list, index=index_list)
df_r2 = pd.DataFrame(d)
df_r2

r2 scores:


,pct_collab_linauth_zscore,pct_collab_linprof_zscore,pct_collab_sqrtauth_zscore,pct_collab_sqrtprof_zscore,pct_collab_unweighted_zscore
avg_PR,0.0013,0.0072,0.0039,0.0087,0.0056
max_PR,0.0003,0.0019,0.0051,0.0075,0.0011
avg_AR,0.0000,0.0056,0.0050,0.0134,0.0183
max_AR,0.0020,0.0029,0.0044,0.0049,0.0003
